In [1]:
import numpy as np
import pandas as pd

import os
import torch
import torchvision
from PIL import Image
import os.path
import torch.optim as optim
import torch.nn as nn

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
import torch.optim as optims
import random
import torchvision.models as models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
dataset_path = "ImageNette"
dataset_exists = os.path.exists(dataset_path)

In [5]:
%pip install --upgrade torchvision
%pip install --upgrade matplotlib

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━

In [7]:
trans = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
              ])

In [ ]:
imagenet_data = torchvision.datasets.Imagenette("ImageNette", download = not dataset_exists, transform = trans)
class_to_indices = list(set(imagenet_data.class_to_idx.values()))

In [ ]:
class_to_indices

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
imagenet_train, imagenet_test = train_test_split(imagenet_data, test_size=0.3, random_state=42)

In [ ]:
class TripletImageDataset(Dataset):
    def __init__(self, dataset, class_to_indices):
        self.dataset = dataset
        self.class_to_indices = class_to_indices

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        anchor_img, positive_idx = self.dataset[idx]
        negative_idx = random.choice([c for c in self.class_to_indices if c != positive_idx])
        positive_img, _ = self.dataset[positive_idx]
        negative_img, _ = self.dataset[negative_idx]
        return anchor_img, positive_img, negative_img


## *SIAMESE NETWORK (PRETRAINED CNN WITH FEED FORWARD)*

In [ ]:
#Pretrained CNN for feature extraction
class pretrained_model(nn.Module):

  def __init__(self, pretrained_CNN, maxpool_ksize, img_trans):
    super(pretrained_model, self).__init__()
    self.pre_CNN = pretrained_CNN
    self.img_trans = img_trans
    self.mp = nn.MaxPool2d(maxpool_ksize, stride=1)
    self.fl = nn.Flatten()

  def preprocess(self, x):
    x = self.img_trans(x)
    return x

  def forward(self, x):
    #Extracting Features
    x = self.pre_CNN(x)

    #2D Maxpooling
    x = self.mp(x)
    x = self.fl(x)

    #L2 Normalisation : TBD

    return x


In [ ]:
class feed_forward(nn.Module):

  def __init__(self, input_size):
    super(feed_forward, self).__init__()
    self.l1 = nn.Linear(input_size, 2048)
    self.l2 = nn.Linear(2048, 1024)
    self.l3 = nn.Linear(1024, 512)
    self.l4 = nn.Linear(512, 256)

    self.dr1 = nn.Dropout(0.3)
    self.dr2 = nn.Dropout(0.3)
    self.dr3 = nn.Dropout(0.25)

    self.relu = nn.ReLU()
    self.elu = nn.ELU()
    self.gelu = nn.GELU()

  def forward(self, x):
    x = self.dr1(self.relu(self.l1(x)))
    x = self.dr2(self.relu(self.l2(x)))
    x = self.dr3(self.relu(self.l3(x)))
    x = self.l4(x)
    return x

In [ ]:
class Siamese(nn.Module):
  def __init__(self, pretrained_CNN, maxpool_ksize, img_trans):
    super(Siamese, self).__init__()
    self.pretrained = pretrained_model(pretrained_CNN, maxpool_ksize, img_trans).to(device)
    self.pretrained.eval()

    input_shape = (1, 3, 224, 224)

    dummy_input = torch.randn(*input_shape).to(device)
    output = self.pretrained(dummy_input)
    feature_size = output.size(1)

    self.ff = feed_forward(feature_size).to(device)
    print(feature_size)

  def forward(self, x):
    with torch.no_grad():
      x = self.pretrained(x)
    # print(x.shape)
    x = self.ff(x)
    return x

In [ ]:
pretrained = models.densenet169(pretrained=True)
maxpool_ksize=6
img_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
layers = list(pretrained.children())[:-1]
pretrained = torch.nn.Sequential(*layers).to(device)

In [ ]:
siamese_model = Siamese(pretrained, maxpool_ksize, img_trans).to(device)

In [ ]:
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-7)

In [ ]:
learning_rate = 1e-4
batch_size = 128
epochs = 50

In [ ]:
triplet_dataset = TripletImageDataset(imagenet_train, class_to_indices)
data_loader = DataLoader(triplet_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
optimizer = optim.AdamW(siamese_model.parameters(), lr=learning_rate, amsgrad=True)

In [ ]:
trainLoss = []

for epoch in range(epochs):
  trainloss = 0.0

  siamese_model.train()
  for batch_idx, (anchor_imgs, positive_imgs, negative_imgs) in enumerate(data_loader):
    optimizer.zero_grad()
    anchor_out = siamese_model(anchor_imgs.to(device))
    positive_out = siamese_model(positive_imgs.to(device))
    negative_out = siamese_model(negative_imgs.to(device))
    loss = triplet_loss(anchor_out, positive_out, negative_out)
    trainloss+=loss.item()
    loss.backward(retain_graph=True)
    optimizer.step()

  trainLoss.append(trainloss/batch_size)
  print("Epoch: ", epoch+1, "  TrainLoss: ", trainloss)


In [ ]:
siamese_model.eval()

embeddings = []
with torch.no_grad():
    for barch_idx, (image, label) in enumerate(imagenet_test):
#         image = img_trans(image).to(device).unsqueeze(0)
        image = siamese_model(image.to(device).unsqueeze(0)).to('cpu')
        embeddings.append(image)

In [ ]:
embeddings_tensor = torch.stack(embeddings).squeeze(1)

In [ ]:
embeddings_tensor.shape

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def knn_grouping(embeddings, k=5):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(embeddings)
    distances, indices = nbrs.kneighbors(embeddings)
    return distances, indices

In [ ]:
distances, indices = knn_grouping(embeddings_tensor.numpy(), 11)

In [ ]:
indices[0]

In [ ]:
from matplotlib import gridspec
import cv2

In [ ]:
def plot_images(indx_list, nearest_list, dataset):
  fig = plt.figure(figsize=(20, 40))
  gs = gridspec.GridSpec(len(indx_list),len(nearest_list[0])+1)
  for i in range(len(indx_list)):
    ax = fig.add_subplot(gs[i,0])
    image, _ = dataset[indx_list[i]]
    image = np.transpose(image, (1,2,0))
    ax.imshow(image)
    ax.axis('off')
    for j in range(1, len(nearest_list[indx_list[i]])):
      ax = fig.add_subplot(gs[i,j+1])
      img, _ = dataset[nearest_list[indx_list[i]][j]]
      img = np.transpose(img, (1,2,0))
      ax.imshow(img)
      ax.axis('off')

In [ ]:
indx_list = random.sample(range(len(indices)), 10)
plot_images(indx_list, indices, imagenet_data)

## *AUTOENCODER*

In [ ]:
class AutoEncoder(nn.Module):

  def __init__(self, input_shape):
    super(AutoEncoder, self).__init__()

    #Encoder Layers
    self.l1 = nn.Linear(input_shape, 2048)
    self.l2 = nn.Linear(2048, 1024)
    self.l3 = nn.Linear(1024, 512)
    self.l4 = nn.Linear(512, 256)
    self.dr1 = nn.Dropout(0.3)
    self.dr2 = nn.Dropout(0.3)
    self.dr3 = nn.Dropout(0.25)
    self.dr4 = nn.Dropout(0.2)

    #Decoder Layers
    self.l5 = nn.Linear(256, 512)
    self.l6 = nn.Linear(512, 1024)
    self.l7 = nn.Linear(1024, 2048)
    self.l8 = nn.Linear(2048, input_shape)
    self.dr5 = nn.Dropout(0.2)
    self.dr6 = nn.Dropout(0.25)
    self.dr7 = nn.Dropout(0.3)

    #Activation Functions
    self.relu = nn.ReLU()
    self.elu = nn.ELU()
    self.gelu = nn.GELU()

  #Encoder Forward Pass
  def encoder(self, x):
    x = self.dr1(self.relu(self.l1(x)))
    x = self.dr2(self.relu(self.l2(x)))
    x = self.dr3(self.relu(self.l3(x)))
    x = self.dr4(self.relu(self.l4(x)))

  #Decoder Forward Pass
  def decoder(self, x):
    x = self.dr5(self.relu(self.l5(x)))
    x = self.dr6(self.relu(self.l6(x)))
    x = self.dr7(self.relu(self.l7(x)))
    x = self.l8(x)
    return x

  #Forward Pass
  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    #Return decoder output as well as bottlneck output
    return decoded, encoded



In [ ]:
# class LinearAutoencoder(nn.Module):
#     def __init__(self, input_dim, encoding_dim):        super(LinearAutoencoder, self).__init__()
#         self.encoder = nn.Linear(input_dim, encoding_dim)
#         self.decoder = nn.Linear(encoding_dim, input_dim)

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x


In [ ]:
# import os
# import requests
# from zipfile import ZipFile

# dataset_url = "http://aws-proserve-data-science.s3.amazonaws.com/geological_similarity.zip"
# filePath = './data_repository/geological_similarity.zip'
# data_directory = './data_repository'

# if not os.path.exists(data_directory):
#     try:
#         os.makedirs(data_directory)
#         print(data_directory," created successfully.")
#     except:
#         print("Unable to create directory at ",data_directory," Please create ",data_directory," manually. Then run this file again.")

# if os.path.exists(filePath):
#     os.remove(filePath)
# else:
#     print("Have to download dataset.")


# r = requests.get(dataset_url, stream = True)
# print('Started downloading dataset...')
# with open(filePath, "wb") as data:
#     for chunk in r.iter_content(chunk_size=1024):
#         # writing one chunk at a time to data file

#         if chunk:
#             print('...',end = ''),
#             data.write(chunk)
# print('Download finished.')
# print('Unzipping File...')
# zf = ZipFile(filePath, 'r')
# zf.extractall('./data_repository/')
# zf.close()
# print('Successfully unzipped file. Ready to run model...')

In [ ]:
import os
from matplotlib.image import imread
import numpy as np


class PreProcessing:

    images_train = np.array([])
    images_test = np.array([])
    labels_train = np.array([])
    labels_test = np.array([])
    unique_train_label = np.array([])
    map_train_label_indices = dict()

    def __init__(self,data_src):
        self.data_src = data_src
        print("Loading Geological Similarity Dataset...")
        self.images_train, self.images_test, self.labels_train, self.labels_test = self.preprocessing(0.9)
        self.unique_train_label = np.unique(self.labels_train)
        self.map_train_label_indices = {label: np.flatnonzero(self.labels_train == label) for label in
                                        self.unique_train_label}
        print('Preprocessing Done. Summary:')
        print("Images train :", self.images_train.shape)
        print("Labels train :", self.labels_train.shape)
        print("Images test  :", self.images_test.shape)
        print("Labels test  :", self.labels_test.shape)
        print("Unique label :", self.unique_train_label)

    def normalize(self,x):
        min_val = np.min(x)
        max_val = np.max(x)
        x = (x - min_val) / (max_val - min_val)
        return x

    def read_dataset(self):
        X = []
        y = []
        for directory in os.listdir(self.data_src):
            # print(directory)
            try:
                i = 0
                for pic in os.listdir(os.path.join(self.data_src, directory)):
                    if i==100:
                      break
                    img = imread(os.path.join(self.data_src, directory, pic))
                    X.append(np.squeeze(np.asarray(img)))
                    y.append(directory)
                    i+=1
            except Exception as e:
                print('Failed to read images from Directory: ', directory)
                print('Exception Message: ', e)
        print('Dataset loaded successfully.')
        return X,y

    def preprocessing(self,train_test_ratio):
        X, y = self.read_dataset()
        labels = list(set(y))
        label_dict = dict(zip(labels, range(len(labels))))
        Y = np.asarray([label_dict[label] for label in y])
        X = [self.normalize(x) for x in X]                                  # normalize images

        shuffle_indices = np.random.permutation(np.arange(len(y)))
        x_shuffled = []
        y_shuffled = []
        for index in shuffle_indices:
            x_shuffled.append(X[index])
            y_shuffled.append(Y[index])

        size_of_dataset = len(x_shuffled)
        n_train = int(np.ceil(size_of_dataset * train_test_ratio))
        print(len(x_shuffled), len(y_s))
        return np.asarray(x_shuffled[0:n_train]), np.asarray(x_shuffled[n_train + 1:size_of_dataset]), np.asarray(
            y_shuffled[0:n_train]), np.asarray(y_shuffled[
                                               n_train + 1:size_of_dataset])


    def get_triplets(self):
        label_l, label_r = np.random.choice(self.unique_train_label, 2, replace=False)
        a, p = np.random.choice(self.map_train_label_indices[label_l],2, replace=False)
        n = np.random.choice(self.map_train_label_indices[label_r])
        return a, p, n

    def get_triplets_batch(self,n):
        idxs_a, idxs_p, idxs_n = [], [], []
        for _ in range(n):
            a, p, n = self.get_triplets()
            idxs_a.append(a)
            idxs_p.append(p)
            idxs_n.append(n)
        return self.images_train[idxs_a,:], self.images_train[idxs_p, :], self.images_train[idxs_n, :]

In [ ]:
dataset = PreProcessing('/content/ImageNette/imagenette2/train')